In [17]:
# https://huggingface.co/docs/transformers/main/model_doc/timesformer
import av
import torch
import numpy as np

from transformers import AutoImageProcessor, TimesformerForVideoClassification
from huggingface_hub import hf_hub_download

np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

# sample 8 frames
indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")

inputs = image_processor(list(video), return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# model predicts one of the 400 Kinetics-400 classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


eating spaghetti


In [18]:
# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = "/home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1001Runs/run23.mp4"

def classify_off_local_video(file_path):
    # replace with your local video file path
    container = av.open(file_path)

    # sample 8 frames
    indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
    video = read_video_pyav(container, indices)

    inputs = image_processor(list(video), return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # model predicts one of the 400 Kinetics-400 classes
    predicted_label = logits.argmax(-1).item()
    print(model.config.id2label[predicted_label])

classify_off_local_video(file_path)

recording music


In [19]:
import os

def classify_videos_in_directory(directory_path):
    # Get a list of all video files in the directory
    video_files = [f for f in os.listdir(directory_path) if f.endswith('.mp4')]

    # Classify each video
    for video_file in video_files:
        file_path = os.path.join(directory_path, video_file)
        print(f"Classifying video: {file_path}")
        classify_off_local_video(file_path)
    
directory_path = "/home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1001Runs"
classify_videos_in_directory(directory_path)

Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1001Runs/run0.mp4
biking through snow
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1001Runs/run1.mp4
busking
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1001Runs/run2.mp4
playing violin
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1001Runs/run3.mp4
pumping gas
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1001Runs/run4.mp4
texting
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1001Runs/run5.mp4
busking
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1001Runs/run6.mp4
busking
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1001Runs/run7.mp4
recording music
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1001Runs/run8.mp4
buski

In [20]:
classify_videos_in_directory("/home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1002Runs")

Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1002Runs/run0.mp4
peeling apples
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1002Runs/run1.mp4
texting
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1002Runs/run2.mp4
playing saxophone
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1002Runs/run3.mp4
finger snapping
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1002Runs/run4.mp4
playing saxophone
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1002Runs/run5.mp4
playing piano
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1002Runs/run6.mp4
shredding paper
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1002Runs/run7.mp4
washing dishes
Classifying video: /home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/AR

In [21]:
classify_videos_in_directory("/home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1003Runs")

FileNotFoundError: [Errno 2] No such file or directory: '/home/blhohmann/miniconda3/envs/myenv/speechviz/data/video/ARIA1003Runs'